In [4]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel
from scipy.optimize import minimize

# 1. Historical Data for Function 3 (Index 2)
# Extracted from the provided files for Weeks 1-3
X = np.array([
    [0.1, 0.1, 0.1],                 # Week 1
    [0.2, 0.2, 0.2],                 # Week 2
    [0.123105, 0.025252, 0.134332]   # Week 3
])

y = np.array([
    -0.09338622073872602,  # Week 1 Output
    -0.16230936451166192,  # Week 2 Output
    -0.1662102640375241    # Week 3 Output
])

# 2. Train Gaussian Process
# Matern kernel allows for non-linear relationships. 
# WhiteKernel handles potential noise in the observations.
kernel = Matern(length_scale=1.0, nu=2.5) + WhiteKernel(noise_level=1e-5)
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, random_state=42)
gp.fit(X, y)

# 3. Define Acquisition Function (Upper Confidence Bound)
# We balance maximizing the mean prediction with exploring high uncertainty (std).
# kappa=2.576 corresponds to roughly 99% confidence exploration.
def acq_func(x, gp, kappa=2.576):
    x = x.reshape(1, -1)
    mean, std = gp.predict(x, return_std=True)
    # We return negative because 'minimize' finds the lowest value
    return -(mean + kappa * std)

# 4. Optimize Next Submission
# We search for inputs between 0 and 1 (positive numbers constraint).
bounds = [(1e-6, 1.0)] * 3
best_x = None
best_val = np.inf

# Run optimization from multiple random starting points to avoid local optima
np.random.seed(42)
for _ in range(50):
    x0 = np.random.uniform(0, 1, 3)
    res = minimize(acq_func, x0, args=(gp,), bounds=bounds, method='L-BFGS-B')
    if res.fun < best_val:
        best_val = res.fun
        best_x = res.x

print("--- Week 4 Prediction ---")
print(f"Recommended Submission for Function 3: {best_x}")
print(f"Predicted Value (Acquisition Score): {-best_val}")

--- Week 4 Prediction ---
Recommended Submission for Function 3: [0.94888554 0.96563203 0.80839735]
Predicted Value (Acquisition Score): -0.018729365600594947
